# 200_smoketest_finetuning_gpt

> The goal of this notebook is to finetune gpt models using a small subset of label comments using OpenAI's api. To see if the project is a viable. 

First let's load the cleaned subset file into a DataFrame.

In [39]:
import pandas as pd
import numpy as np
import openai

In [40]:
comments_df = pd.read_csv("../data/cleaned/channel_subset_with_comments.csv", index_col='comment_id')
comments_df

,channel_id,channel_title,affiliation,video_id,video_title,video_description,comment_text
comment_id,,,,,,,
Ugxtc_P8-7EmyHXAaF14AaABAg,UCAlVRoYjDbgLx7j6O2N8-2g,Men Are Good!,R,-T9vHzMdir4,Women Rally to Close the Gender Death Gap. NOT.,"Paul, Tom, and Janice discuss a New York Times...",Men's generally poorer health from stress caus...
UgyhGIdaUug-1Ciyfbh4AaABAg,UCAlVRoYjDbgLx7j6O2N8-2g,Men Are Good!,R,-T9vHzMdir4,Women Rally to Close the Gender Death Gap. NOT.,"Paul, Tom, and Janice discuss a New York Times...","Is anyone on gab? I can't find people there, p..."
UgwGR7A9eYedizYeCGZ4AaABAg,UCAlVRoYjDbgLx7j6O2N8-2g,Men Are Good!,R,-T9vHzMdir4,Women Rally to Close the Gender Death Gap. NOT.,"Paul, Tom, and Janice discuss a New York Times...",Life sucks doesn't it ladies. He just up and d...
UgyIq8h56bQaUI-O55l4AaABAg,UCAlVRoYjDbgLx7j6O2N8-2g,Men Are Good!,R,-T9vHzMdir4,Women Rally to Close the Gender Death Gap. NOT.,"Paul, Tom, and Janice discuss a New York Times...",I have seen figures that show less death in th...
UgzLSxmvmakux6GzJG14AaABAg,UCAlVRoYjDbgLx7j6O2N8-2g,Men Are Good!,R,-T9vHzMdir4,Women Rally to Close the Gender Death Gap. NOT.,"Paul, Tom, and Janice discuss a New York Times...","""Waiting for the call for women to protect men..."
...,...,...,...,...,...,...,...
UgzziTnfC9kiB5qsyyF4AaABAg.9IAyyudOaoc9IDbTcSKMxR,UCr_dQZ0irQdRiwVatkuHPcA,UK Column,L,2yoqlbn9Ts0,UK Column News - 6th January 2021,"Brian Gerrish, Mike Robinson, Alex Thomson and...",Their are lies damn lies and British politicia...
UgzziTnfC9kiB5qsyyF4AaABAg.9IAyyudOaoc9ICpFxtI6ro,UCr_dQZ0irQdRiwVatkuHPcA,UK Column,L,2yoqlbn9Ts0,UK Column News - 6th January 2021,"Brian Gerrish, Mike Robinson, Alex Thomson and...",So many people with large family & friend netw...
UgwWdIXwRecT8RRGoZ14AaABAg.9IAyY08RTtV9IB9FsByn5z,UCr_dQZ0irQdRiwVatkuHPcA,UK Column,L,2yoqlbn9Ts0,UK Column News - 6th January 2021,"Brian Gerrish, Mike Robinson, Alex Thomson and...",wanna discuss clap trap Brian?


### Let's get the openAI api set up

In [41]:
with open("C:/Users/danie/OneDrive/Desktop/openai_youtube_api_key.txt") as f:
    api_key = f.readline()

openai.api_key = api_key

### Next, create the dictionary of fine tuning training examples.
> This may involve a number of steps including uploading the file to the OpenAI server.